# D-MDSP - Exact algorithm for finding the minimum dominating set in directed graphs (Sink)

### Importación de librerias

In [1]:
import os
import gurobipy as gp

### Definición de funciones

In [2]:
def read_adjacency_matrix(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    adjacency_matrix = []
    for line in lines:
        row = [int(x) for x in line.strip().split()]
        adjacency_matrix.append(row)
    return adjacency_matrix

In [3]:
def find_minimum_dominating_set_reversed(adjacency_matrix, time_limit):
    num_vertices = len(adjacency_matrix)
    model = gp.Model("MinimumDominatingSetReversed")

    # Add variables: 1 if vertex is in the dominating set, 0 otherwise
    dominating_vars = model.addVars(num_vertices, vtype=gp.GRB.BINARY, name="dominating")

    # Add constraints: each non-dominated vertex is dominated by at least one vertex in the dominating set
    for i in range(num_vertices):
        neighbors = [j for j in range(num_vertices) if adjacency_matrix[i][j] == 1]
        model.addConstr(dominating_vars.sum(neighbors) >= 1)

    # Set the objective to minimize the size of the dominating set
    model.setObjective(dominating_vars.sum(), sense=gp.GRB.MINIMIZE)

    # disable gurobi output
    model.setParam('OutputFlag', 0)
    
    # Set time limit
    model.setParam('TimeLimit', time_limit)
    
    # Optimize the model
    model.optimize()

    # Retrieve and print the results
    dominating_set = [i for i in range(num_vertices) if dominating_vars[i].X > 0.5]

    return dominating_set, model.Runtime, model.MIPGap, model.status == gp.GRB.OPTIMAL

In [4]:
def execute_wmdsp(folder, time_limit):
    with open("../src/main/resources/results/sink-" + folder + ".csv", 'a') as f:
        f.write("filename;size;dominating_set;runtime;mipgap;optimal\n")

    # for each file in the folder
    for filename in os.listdir("../src/main/resources/graphs/" + folder + "/"):
        # read the adjacency matrix
        adjacency_matrix = read_adjacency_matrix(os.path.join("../src/main/resources/graphs/" + folder + "/", filename))

        # find the minimum dominating set
        dominating_set, runtime, mipgap, optimal = find_minimum_dominating_set_reversed(adjacency_matrix, time_limit)

        # export the results with columns named "filename, size, dominating_set, runtime, mipgap, optimal" to a csv file and split by ";"
        with open("../src/main/resources/results/sink-" + folder + ".csv", 'a') as f:
            f.write(filename + ";" + str(len(dominating_set)) + ";" + str(dominating_set) + ";" + str(runtime) + ";" + str(mipgap) + ";" + str(optimal) + "\n")

### Definición de parámetros y ejecución

In [5]:
execute_wmdsp("random", 1800)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-04
